# Dependências

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
# !pip install gcloud
# !gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

# Tratamento

In [ ]:
# Load civil servant data from CSV file
df = pd.read_csv('siape_vinculos.csv', sep=';')

# Rename columns for clarity
df = df.rename(columns={
    'sexo': 'genero',
    'ano_ing_spub': 'ano_ingresso',
    'raca_grupo': 'raca',
    'servidores': 'total_vinculo'
})

# Function to standardize gender labels
def transformar(nome):
    nome = re.sub(r"\bFem\b", "Mulheres", nome)
    nome = re.sub(r"\bMas\b", "Homens", nome)
    return nome

# Apply gender standardization
df['genero'] = df['genero'].apply(transformar)

# Rename additional columns
df = df.rename(columns={"raca": "cor_raca", "total_vinculo": "quantidade_vinculos"})

# Create combined gender-race column
df['genero_raca'] = df['genero'] + ' ' + df['cor_raca']

# Select relevant columns
df = df[['ano_ingresso', 'cor_raca', 'genero', 'genero_raca', 'quantidade_vinculos']]

# Function to standardize gender-race labels
def transformar(nome):
    nome = re.sub(r"\bMulheres Branca\b", "Mulheres Brancas", nome)
    nome = re.sub(r"\bMulheres Negra\b", "Mulheres Negras", nome)
    nome = re.sub(r"\bMulheres Não informada\b", "Mulheres sem informação de raça", nome)
    nome = re.sub(r"\bMulheres Outros\b", "Mulheres com outras raças", nome)
    nome = re.sub(r"\bHomens Branca\b", "Homens Brancos", nome)
    nome = re.sub(r"\bHomens Negra\b", "Homens Negros", nome)
    nome = re.sub(r"\bHomens Não informada\b", "Homens sem informação de raça", nome)
    nome = re.sub(r"\bHomens Outros\b", "Homens com outras raças", nome)
    return nome

# Apply gender-race standardization
df['genero_raca'] = df['genero_raca'].apply(transformar)

# Function to standardize gender labels to formal format
def transforme(nome):
    nome = re.sub(r"\bMulheres\b", "Feminino", nome)
    nome = re.sub(r"\bHomens\b", "Masculino", nome)
    return nome

# Apply formal gender standardization
df['genero'] = df['genero'].apply(transforme)

# Clean and convert numeric values
df['quantidade_vinculos'] = df['quantidade_vinculos'].str.replace(',', '.')
df['quantidade_vinculos'] = pd.to_numeric(df['quantidade_vinculos'], errors='coerce')
df['quantidade_vinculos'] = df['quantidade_vinculos'].astype('Int64')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ano_ingresso         163 non-null    int64 
 1   cor_raca             163 non-null    object
 2   genero               163 non-null    object
 3   genero_raca          163 non-null    object
 4   quantidade_vinculos  163 non-null    Int64 
dtypes: Int64(1), int64(1), object(3)
memory usage: 6.7+ KB


# Upload

In [ ]:
# Define schema for BigQuery table
schema = [
    bigquery.SchemaField('ano_ingresso','INTEGER',description='Ano referente ao ingresso da pessoa na profissão'),
    bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada '),
    bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
    bigquery.SchemaField('genero_raca','STRING',description='Gênero e raça da pessoa observada'),
    bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Número total de vinculos observados')
]

# Initialize BigQuery client
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('perfil_remuneracao')

# Create table reference
table_ref = dataset_ref.table('SIAPE_ativos_federal_raca_sexo_ingresso_v1')

# Configure load job
job_config = bigquery.LoadJobConfig(schema=schema)

# Execute load job
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()